In [ ]:
import tensorflow as tf
import os
from tensorflow.python.tools import freeze_graph

slim = tf.contrib.slim
# from tensorflow.contrib.slim.nets import vgg
from tensorflow.contrib.slim.python.slim.nets import resnet_v2

model_path = "/home/nishome/jjzhao/repos/vgg16/models/vgg_16.ckpt"  # 设置model的路径


def main():
    tf.reset_default_graph()

#     input_node = tf.placeholder(tf.float32, shape=(240, 240, 3))
#     input_node = tf.expand_dims(input_node, 0)
    input_node = tf.random_normal([1, 224, 224, 3])
    with slim.arg_scope(resnet_v2.resnet_arg_scope(is_training=False)):
        net, _ = resnet_v2.resnet_v2_50(input_node, 1000)
    flow = tf.cast(flow, tf.uint8, 'output')  # 设置输出类型以及输出的接口名字，为了之后的调用pb的时候使用
  
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, model_path)

        # 保存图
        tf.train.write_graph(sess.graph_def, './output_model/pb_model', 'model.pb')
        # 把图和参数结构一起
        freeze_graph.freeze_graph('output_model/pb_model/model.pb',
                                  '',
                                  False,
                                  model_path,
                                  'output',
                                  'save/restore_all',
                                  'save/Const:0',
                                  'output_model/pb_model/frozen_model_vgg_16.pb',
                                  False,
                                  "")

    print("done")


if __name__ == '__main__':
    main()

In [ ]:
# resnet50:
#   # has only checkpoint format
#   disabled: true
#   url: http://download.tensorflow.org/models/official/resnet50_2017_11_30.tar.gz
#   model: fixme
#   input_get: get_beach
#   inputs:
#     "input:0": [1, 416, 416, 3]
#   outputs:

import os
import tensorflow as tf
import numpy as np
from tensorflow.python.tools import freeze_graph
from tensorflow.python.framework import graph_util
import os.path
import argparse


if __name__ == '__main__':
    x = tf.random_normal([1, 416, 416, 3])
    # model_output = darknet(x)

    saver = tf.train.Saver()

    output_graph = "resnet50.pb"
    #  output name : 'conv_dec/BiasAdd:0' conv_dec/BiasAdd
    #  input name : x : 'random_normal:0'

    with tf.Session() as sess:

        # 必须先restore模型才能打印shape;导入模型时，上面每层网络的name不能修改，否则找不到
        saver.restore(sess, "model.ckpt")
        # print(sess.run(model_output).shape)  # (1,13,13,425)

        # 得到当前的图的 GraphDef 部分，通过这个部分就可以完成重输入层到输出层的计算过程
        graph_def = tf.get_default_graph().as_graph_def()

        # 模型持久化，将变量值固定
        output_graph_def = graph_util.convert_variables_to_constants(
            sess,
            graph_def,
            ["output"]  # 需要保存节点的名字
        )
        with tf.gfile.GFile(output_graph, "wb") as f:  # 保存模型
            f.write(output_graph_def.SerializeToString())  # 序列化输出

